In [1]:
import Ipynb_importer
from a_basic_quant import *
from b_model import *
from c_train_and_test import *
from d_post_training_quantize import *

importing Jupyter notebook from a_basic_quant.ipynb
importing Jupyter notebook from b_model.ipynb
importing Jupyter notebook from c_train_and_test.ipynb
importing Jupyter notebook from d_post_training_quantize.ipynb


In [2]:
a = torch.randn((64,1,28,28))
a.shape

torch.Size([64, 1, 28, 28])

In [3]:
conv = torch.nn.Conv2d(1, 40, 3, 1)
res = conv(a)

In [4]:
min_a, max_a = calcu_max_and_min(a, None, None, False)
scale_a, zero_point_a = calcu_scale_and_zeropoint(min_a, max_a, 8, False)
q_a = quantize_tensor(a, scale_a, zero_point_a).int()
dq_a = dequantize_tensor(q_a, scale_a, zero_point_a)
print(a[0][0:5][0][0])
print(dq_a[0][0:5][0][0])

tensor([-1.6115e+00,  2.5057e+00, -2.4694e-03,  6.5107e-01, -6.8049e-01,
         1.5487e-01, -2.9745e-01,  1.8995e+00,  1.5417e+00,  8.7582e-01,
         3.0330e-01,  1.0632e+00,  5.2285e-01, -1.2756e-01, -1.0106e+00,
        -4.3086e-01,  2.0793e-01,  1.3761e+00, -4.0121e-01,  9.5955e-01,
        -1.8500e+00,  1.0811e+00, -1.2179e+00,  9.6849e-01,  2.8365e-01,
        -2.5091e-01,  1.5151e+00, -1.1316e+00])
tensor([-1.6330,  2.4995, -0.0333,  0.6332, -0.6999,  0.1333, -0.2999,  1.8663,
         1.5330,  0.8665,  0.2999,  1.0331,  0.4999, -0.1333, -1.0331, -0.4332,
         0.2000,  1.3664, -0.4332,  0.9331, -1.8663,  1.0664, -1.2331,  0.9665,
         0.2666, -0.2666,  1.4997, -1.1331])


In [5]:
w = conv.weight.data
min_w, max_w = calcu_max_and_min(w, None, None, True)
scale_w, zero_point_w = calcu_scale_and_zeropoint(min_w, max_w, 8, True)
q_w = quantize_tensor(w, scale_w, zero_point_w,8,False, True)
dq_w = dequantize_tensor(q_w, scale_w, zero_point_w, True)
print(w[0][0:5][0][0])
print(dq_w[0][0:5][0][0])

tensor([0.1926, 0.0502, 0.1333])
tensor([0.1926, 0.0502, 0.1333])


In [6]:
min_res, max_res = calcu_max_and_min(res, None, None, False)
scale_res, zero_point_res = calcu_scale_and_zeropoint(min_res, max_res, 8, False)
q_res = quantize_tensor(res, scale_res, zero_point_res).int()
dq_res = dequantize_tensor(q_res, scale_res, zero_point_res)
print(res[0][0:5][0][0])
print(dq_res[0][0:5][0][0])

tensor([-0.1765,  0.8441, -0.7927,  0.0257, -1.1507,  0.8137,  0.6025,  0.8553,
         1.0318,  0.0679,  0.2848,  0.0923, -0.3824, -0.3483,  0.0204,  0.6378,
         0.6959,  0.0553, -0.9178, -0.6067, -0.9119, -0.0210, -0.4007, -0.1225,
         0.1902,  0.4652], grad_fn=<SelectBackward>)
tensor([-0.2036,  0.8435, -0.8144,  0.0000, -1.1634,  0.7853,  0.5817,  0.8435,
         1.0180,  0.0582,  0.2618,  0.0873, -0.4072, -0.3490,  0.0000,  0.6108,
         0.6690,  0.0291, -0.9307, -0.6108, -0.9307, -0.0291, -0.4072, -0.1454,
         0.1745,  0.4363])


In [7]:
q_w[0].view(1,1,3,3).shape

torch.Size([1, 1, 3, 3])

In [8]:
conv.bias[0].shape

torch.Size([])

In [9]:
x = q_a - zero_point_a
y = F.conv2d(input = x, weight = q_w[0].view(1,1,3,3), bias=conv.bias[0].view(1),\
            stride=conv.stride,
            padding=conv.padding,
            dilation=conv.dilation,
            groups=conv.groups)
y.shape

torch.Size([64, 1, 26, 26])

In [10]:
conv.weight.data.shape

torch.Size([40, 1, 3, 3])

In [11]:
q_w.shape

torch.Size([40, 1, 3, 3])

In [12]:
torch.tensor(zero_point_w)[23]

tensor(179., dtype=torch.float64)

In [13]:
for i in range(len(zero_point_w)):
    conv.weight.data[i] = q_w[i] - zero_point_w[i]

In [14]:
x_2 = q_a - zero_point_a
y_2 = conv(x_2)
y_2.shape

torch.Size([64, 40, 26, 26])

In [15]:
M = [scale_w[i] * scale_a / scale_res for i in range(len(scale_w))]
len(M)

40

In [16]:
for i in range(len(M)):
    y_2[:,i,:,:] = y_2[:,i,:,:] * M[i]
y_2[0][0:5][0][0]

tensor([-11.3463,  24.0449, -32.4485,  -4.1960, -44.3211,  22.9628,  15.7608,
         24.2389,  30.6368,  -2.5505,   4.6794,  -2.1993, -18.5277, -17.1324,
         -4.3382,  16.7879,  18.6787,  -3.4253, -36.9573, -26.2078, -36.6000,
         -5.8291, -19.2183,  -9.5266,   1.4173,  11.0966],
       grad_fn=<SelectBackward>)

In [17]:
z_2 = y_2 +zero_point_res
z_2[0][0:5][0][0]

tensor([111.6537, 147.0449,  90.5515, 118.8040,  78.6789, 145.9628, 138.7608,
        147.2389, 153.6368, 120.4494, 127.6794, 120.8007, 104.4723, 105.8676,
        118.6618, 139.7879, 141.6787, 119.5747,  86.0427,  96.7922,  86.4000,
        117.1709, 103.7817, 113.4734, 124.4173, 134.0966],
       grad_fn=<SelectBackward>)

In [18]:
dz_2 = dequantize_tensor(z_2, scale_res, zero_point_res)
dz_2[0][0:5][0][0]

tensor([-0.3300,  0.6994, -0.9438, -0.1220, -1.2891,  0.6679,  0.4584,  0.7050,
         0.8911, -0.0742,  0.1361, -0.0640, -0.5389, -0.4983, -0.1262,  0.4883,
         0.5433, -0.0996, -1.0749, -0.7623, -1.0645, -0.1695, -0.5590, -0.2771,
         0.0412,  0.3227], grad_fn=<SelectBackward>)

In [19]:
a = torch.randn((64,1,28,28))
a.shape

torch.Size([64, 1, 28, 28])

In [20]:
conv = torch.nn.Conv2d(1, 40, 3, 1)
b = conv(a)

In [21]:
qconv = QConv2d(conv, True, True, 8, True)

In [22]:
q_b = qconv(a)

In [23]:
qconv.freeze()

In [24]:
q_x = qconv.q_in.quantize_tensor(a)
q_x = qconv.quantize_inference(q_x)

In [25]:
out = qconv.q_out.dequantize_tensor(q_x)
out[0][0][:5][0]

tensor([ 0.0259,  0.4147,  0.0518,  0.0000, -0.0778,  0.0778,  0.0778, -0.1555,
        -0.1555, -0.3110,  0.7776, -0.7257,  0.7257, -0.9072,  0.1037,  1.2182,
         0.0000, -0.5702,  0.6221,  0.9849, -0.3110,  0.3110, -0.1555, -0.6480,
         0.2074,  0.9072], grad_fn=<SelectBackward>)

In [26]:
b[0][0][:5][0]

tensor([ 0.0120,  0.4176,  0.0365, -0.0124, -0.0794,  0.0783,  0.0720, -0.1731,
        -0.1704, -0.3277,  0.7650, -0.7318,  0.7227, -0.9066,  0.1060,  1.2092,
        -0.0069, -0.5866,  0.6306,  0.9811, -0.3208,  0.3115, -0.1566, -0.6419,
         0.1979,  0.9120], grad_fn=<SelectBackward>)